In [ ]:
import matplotlib.pyplot as plt
import csv
import os

import villas.dataprocessing.readtools as rt
import villas.dataprocessing.timeseries as ts
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.validationtools as vt

### Read DPsim files

In [ ]:
# Set path to file
path = '../logs/WSCC-9bus_cosim_extension/'
#path = '../logs/BackupComparison/30M-10M/'
fileName = 'WSCC-9bus_cosim_extension'
resultFile = path + fileName + '.csv'

# Returns list of timeseries-objects
ts_dpsim = rt.read_timeseries_dpsim(resultFile)

### Read DistAIX files

In [ ]:
# Change file to distaix path
fileName = '3_4'
resultFile = path + fileName + '.csv'
resultFileFitted = path + fileName + '_temp.csv'

As DistAIX uses different headers, those have to be adapted to be used by villas.dataprocessing

In [ ]:
with open(resultFile, 'r') as inFile, open(resultFileFitted, 'w') as outFile:
    read = csv.reader(inFile)
    write = csv.writer(outFile)
    
    new_header = []
    row = next(read)

    new_header.append('time')
    for element in row[1:]:
        for idx,char in enumerate(element):
            if char == '[':
                break
        if idx is len(element)-1:
            idx += 1
        new_header.append(element[:idx].strip())
        
    write.writerow(new_header)
    for row in read:
        write.writerow(row)

In [ ]:
ts_distaix = rt.read_timeseries_csv(resultFileFitted)
os.remove(resultFileFitted)

### Compare the wished timeseries created by DPsim and DistAIX and calculate the error

In [ ]:
key_dpsim = 'I_Load3'
key_distaix = 'i2'

ts1 = ts_dpsim[key_dpsim]
ts2 = ts_distaix[key_distaix]

####### Temporary test/workaround as I am not sure, why currents are negative?!
####### Is this, because inflow currents are positive and we compare the
####### inflow in DPsim's load with the outgoing current of DistAIX's cable??
for count, value in enumerate(ts2.values.real):
    ts2.values.real[count] = -value 
for count, value in enumerate(ts2.values.imag):
    ts2.values.imag[count] = -value
    
rmse = ts1.rmse(ts1,ts2)
rmse_norm = ts1.norm_rmse(ts1, ts2)

print('')
print('####################################')
print('RMSE: ' + str(rmse))
print('RMSE_norm: ' + str(rmse_norm))
print('####################################')
print('')
print('')

diff = ts1.diff('diff', ts1, ts2)
print('###########################')
print('Real values:')
print('DPsim: ' + str(ts1.values.real))
print('DistAIX: ' + str(ts2.values.real))
print('###########################')
print('')
print('')

print('###########################')
print('Imag values:')
print('DPsim: ' + str(ts1.values.imag))
print('DistAIX: ' + str(ts2.values.imag))
print('###########################')
print('')
print('')
#plt.plot(diff.values.real)

#### Time series plots


In [ ]:
print('Timeseries plots')
pt.set_timeseries_labels([ts1, ts2], ['DPsim: ' + key_dpsim, 'DistAIX: ' + key_distaix])
pt.plot_timeseries(1,[ts1, ts2])